# Performance Optimization

This notebook teaches you how to optimize simulation performance for real-time applications.

## What You'll Learn

1. Identifying performance bottlenecks
2. Optimizing costmap resolution and visualization
3. Reducing visualization overhead
4. Optimizing sensor processing
5. Profiling and measuring performance

## Prerequisites

- Understanding of simulations (see [Building Simulations](../building/building_simulation.ipynb))
- Understanding of costmaps (see [Costmap Tutorial](costmap_tutorial.ipynb))
- Basic profiling knowledge

In [ ]:
import sys

sys.path.insert(0, '../../src')

import time

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    Car,
    CarState,
    GridCostmap,
    GroundTruthMap,
    LiDARSensor,
    PerceivedMap,
    PurePursuitController,
    Track,
    TrackPlanner,
)

## Step 1: Profiling Simulation Performance

In [ ]:
def profile_simulation_step(car, track, ground_truth_map, planner, controller, costmap, num_iterations=100):
    """
    Profile a single simulation step to identify bottlenecks.
    """
    times = {
        "sensing": [],
        "costmap_update": [],
        "planning": [],
        "control": [],
        "car_update": [],
    }

    for i in range(num_iterations):
        # Sensing
        t0 = time.time()
        perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
        times["sensing"].append(time.time() - t0)

        # Costmap update
        t0 = time.time()
        costmap.update(perception_data, car.state)
        times["costmap_update"].append(time.time() - t0)

        # Planning
        t0 = time.time()
        plan = planner.plan(car.state, perception_data=perception_data, costmap=costmap)
        times["planning"].append(time.time() - t0)

        # Control
        t0 = time.time()
        control = controller.compute_control(car.state, perception_data=perception_data, costmap=costmap, plan=plan)
        times["control"].append(time.time() - t0)

        # Car update
        t0 = time.time()
        car.update(0.1, acceleration=control["acceleration"], steering_rate=control["steering_rate"])
        times["car_update"].append(time.time() - t0)

    # Calculate averages
    results = {}
    for key, values in times.items():
        results[key] = {
            "mean": np.mean(values) * 1000,  # Convert to ms
            "max": np.max(values) * 1000,
            "min": np.min(values) * 1000,
        }

    return results

# Setup
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(ground_truth_map)

start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0))

lidar = LiDARSensor(ground_truth_map, perceived_map, max_range=40.0, name="lidar")
car.add_sensor(lidar)

planner = TrackPlanner(track, lookahead_distance=50.0)
controller = PurePursuitController(lookahead_distance=10.0, target_velocity=8.0)
costmap = GridCostmap(width=60.0, height=60.0, resolution=2.0, frame="ego")

# Profile
print("Profiling simulation performance...")
results = profile_simulation_step(car, track, ground_truth_map, planner, controller, costmap, num_iterations=50)

print("\nPerformance Results (milliseconds):")
print("=" * 50)
for key, stats in results.items():
    print(f"{key:20s}: mean={stats['mean']:6.2f}ms, max={stats['max']:6.2f}ms, min={stats['min']:6.2f}ms")

total_time = sum([stats['mean'] for stats in results.values()])
print(f"\nTotal step time: {total_time:.2f}ms")
print(f"Max frequency: {1000/total_time:.1f} Hz")

## Step 2: Optimizing Costmap Resolution

In [ ]:
# Compare different resolutions
resolutions = [0.5, 1.0, 2.0, 3.0]
update_times = []

for res in resolutions:
    costmap = GridCostmap(width=60.0, height=60.0, resolution=res, frame="ego")

    # Time costmap update
    perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})

    times = []
    for _ in range(20):
        t0 = time.time()
        costmap.update(perception_data, car.state)
        times.append((time.time() - t0) * 1000)

    update_times.append(np.mean(times))
    print(f"Resolution {res:4.1f}m: {np.mean(times):6.2f}ms (cells: {costmap.width_pixels * costmap.height_pixels})")

# Plot results
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(resolutions, update_times, 'o-', linewidth=2, markersize=8)
ax.set_xlabel("Resolution (m)")
ax.set_ylabel("Update Time (ms)")
ax.set_title("Costmap Update Time vs Resolution")
ax.grid(True, alpha=0.3)
plt.show()

print("\n✓ Resolution comparison complete!")
print("  Higher resolution = more cells = slower updates")

## Step 3: Visualization Optimization

In [ ]:
from simple_autonomous_car.visualization import plot_costmap

# Compare visualization methods
costmap = GridCostmap(width=60.0, height=60.0, resolution=2.0, frame="ego")
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
costmap.update(perception_data, car.state)

# Time visualization
fig, ax = plt.subplots(figsize=(8, 8))

times = []
for _ in range(10):
    ax.clear()
    t0 = time.time()
    plot_costmap(costmap, car.state, ax=ax, show_car=True, alpha=0.5)
    times.append((time.time() - t0) * 1000)

print(f"✓ Visualization time: {np.mean(times):.2f}ms (std: {np.std(times):.2f}ms)")
print("  Using imshow (optimized) for fast rendering")

## Step 4: Optimization Tips

In [ ]:
print("Performance Optimization Tips:")
print("=" * 60)
print("")
print("1. Costmap Resolution:")
print("   - Use 2.0m or higher for real-time applications")
print("   - Lower resolution = fewer cells = faster updates")
print("")
print("2. Visualization:")
print("   - Update visualization every 5-10 steps, not every step")
print("   - Use imshow instead of individual rectangles")
print("   - Only show costmap in ego view (not world view)")
print("")
print("3. Sensor Processing:")
print("   - Reduce sensor resolution if needed")
print("   - Limit sensor range")
print("   - Process sensors in parallel if possible")
print("")
print("4. Planning:")
print("   - Use simpler planners for real-time")
print("   - Cache plans when possible")
print("   - Limit plan lookahead distance")
print("")
print("5. General:")
print("   - Profile your code to find bottlenecks")
print("   - Optimize the slowest components first")
print("   - Consider using numba or cython for hot paths")

## Summary

You've learned:

1. ✅ **Profiling**: How to measure performance
2. ✅ **Costmap optimization**: Resolution vs performance trade-off
3. ✅ **Visualization optimization**: Using imshow for speed
4. ✅ **Best practices**: Tips for real-time performance

### Key Concepts

- **Resolution trade-off**: Higher detail vs faster processing
- **Visualization overhead**: Can be significant bottleneck
- **Profiling**: Measure before optimizing
- **Bottlenecks**: Focus on slowest components

### Next Steps

- Implement parallel sensor processing
- Use numba for hot paths
- Build adaptive resolution costmaps
- Implement level-of-detail visualization